In [ ]:
from src import *

import numpy as np
import torch

import time

%matplotlib inline


# CUDA

In [ ]:
torch.manual_seed(1)
torch.cuda.manual_seed(1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('PyTorch is using', device)

# Parameters

In [ ]:
from configs import *

# Main

### Dataset

In [ ]:
transforms = transforms.Compose([
    transforms.Resize(size=(resize_h, resize_w)),
    transforms.ToTensor(),    
])
dataset = CustomDataset(dataset_name=dataset_name, transforms=transforms)
train_loader = dataset.get_dataloader(is_train=True,
                                      batch_size=batch_size,
                                      shuffle=True,
                                      num_workers=num_workers,
                                      prefetch_factor=prefetch_factor)
test_loader = dataset.get_dataloader(is_train=False,
                                     batch_size=batch_size,
                                     shuffle=False,
                                     num_workers=num_workers,
                                     prefetch_factor=prefetch_factor)

### Model

In [ ]:
input_dim = resize_h, resize_w, input_ch
model = VAE(input_dim, channels, num_z).to(device)
print(model)

criterion = ELBO(input_dim)
optimizer = optim.Adam(model.parameters(), lr=init_lr)
scheduler = optim.lr_scheduler.LambdaLR(optimizer=optimizer,
                                        lr_lambda=lambda epoch: lr_decay ** epoch)

In [ ]:
# Learn
total_time = 0
train_losses = {"ELBO": []}
test_losses = {"ELBO": [], "MSE":[]}

for epoch in range(1, epochs + 1):
    # generate and visualize
    samples, recons = reconstruct(model, test_loader, device)
    visualize_imgs(samples, recons)
    
    # train
    start_time = time.time()
    train_loss = train(epoch,
                       model, train_loader,
                       criterion, optimizer, scheduler,
                       device)
    end_time = time.time()
    train_losses["ELBO"].append(train_loss)
    dt = end_time - start_time
    total_time += dt

    # test
    test_elbo_loss, test_mse_loss = test(epoch,
                                         model, test_loader,
                                         criterion,
                                         device)
    test_losses["ELBO"].append(test_elbo_loss)
    test_losses["MSE"].append(test_mse_loss)
    
    print(f'Epoch {epoch} / {epochs} in {dt:.2f} secs')
    print(f'Train loss[ELBO] {train_loss:.4f}, Test loss[ELBO] {test_elbo_loss:.4f}, Test loss[MSE] {test_mse_loss:.5f}')

print('Train loss[ELBO]:', train_losses["ELBO"])
print('Test loss[ELBO]:', test_losses["ELBO"])
print('Test loss[MSE]:', test_losses["MSE"])
print(f'Average {total_time / epochs:.2f} secs per epoch consumed')
print(f'Total {total_time:.2f} secs consumed')

# Save results

In [ ]:
import os
import pandas as pd

save_dir = "./results"
save_dir = os.path.realpath(save_dir)
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

# save model
state_dict_path = os.path.join(save_dir, "vae_state_dict.pth")
state_dict = model.to('cpu').state_dict()
torch.save(state_dict, state_dict_path)

# save results
results = {
    "TrainLoss-ELBO" : train_losses["ELBO"],
    "TestLoss-ELBO"  : test_losses["ELBO"],
    "TestLoss-MSE"   : test_losses["MSE"],
    "TrainTime"      : elapsed_times,
}
results_path = os.path.join(save_dir, "vae_training_results.csv")
results_df = pd.DataFrame(results).reset_index(drop=True)
results_df.to_csv(results_path)